# NLP Basics: Implementing a pipeline to clean text

### Pre-processing text data

Cleaning up the text data is necessary to highlight attributes that you're going to want your machine learning system to pick up on. Cleaning (or pre-processing) the data typically consists of a number of steps:
1. **Remove punctuation**
2. **Tokenization**
3. **Remove stopwords**
4. Lemmatize/Stem

The first three steps are covered in this chapter as they're implemented in pretty much any text cleaning pipeline. Lemmatizing and stemming are covered in the next chapter as they're helpful but not critical.

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t', header=None)
data.columns = ['label', 'body_text']

data.head()

label  \
0   ham   
1  spam   
2   ham   
3   ham   
4   ham   

                                                                                             body_text  
0  I've been searching for the right words to thank you for this breather. I promise i wont take yo...  
1  Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...  
2                                        Nah I don't think he goes to usf, he lives around here though  
3                        Even my brother is not like to speak with me. They treat me like aids patent.  
4                                                                  I HAVE A DATE ON SUNDAY WITH WILL!!

In [2]:
# What does the cleaned version look like?
data_cleaned = pd.read_csv("SMSSpamCollection_cleaned.tsv", sep='\t')
data_cleaned.head()

label  \
0   ham   
1  spam   
2   ham   
3   ham   
4   ham   

                                                                                             body_text  \
0  I've been searching for the right words to thank you for this breather. I promise i wont take yo...   
1  Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...   
2                                        Nah I don't think he goes to usf, he lives around here though   
3                        Even my brother is not like to speak with me. They treat me like aids patent.   
4                                                                  I HAVE A DATE ON SUNDAY WITH WILL!!   

                                                                                      body_text_nostop  
0  ['ive', 'searching', 'right', 'words', 'thank', 'breather', 'promise', 'wont', 'take', 'help', '...  
1  ['free', 'entry', '2', 'wkly', 'comp', 'win', 'fa', 'cup', 'final', 'tkts', '21st', 'may', '2005...  
2                                 ['nah', 'dont', 'think', 'goes', 'usf', 'lives', 'around', 'though']  
3                              ['even', 'brother', 'like', 'speak', 'treat', 'like', 'aids', 'patent']  
4                                                                                   ['date', 'sunday']

### Remove punctuation

In [3]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [4]:
"I like NLP." == "I like NLP"

False

In [5]:
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

data['body_text_clean'] = data['body_text'].apply(lambda x: remove_punct(x))

data.head()

label  \
0   ham   
1  spam   
2   ham   
3   ham   
4   ham   

                                                                                             body_text  \
0  I've been searching for the right words to thank you for this breather. I promise i wont take yo...   
1  Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...   
2                                        Nah I don't think he goes to usf, he lives around here though   
3                        Even my brother is not like to speak with me. They treat me like aids patent.   
4                                                                  I HAVE A DATE ON SUNDAY WITH WILL!!   

                                                                                       body_text_clean  
0  Ive been searching for the right words to thank you for this breather I promise i wont take your...  
1  Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005 Text FA to 87121 to receive e...  
2                                          Nah I dont think he goes to usf he lives around here though  
3                          Even my brother is not like to speak with me They treat me like aids patent  
4                                                                    I HAVE A DATE ON SUNDAY WITH WILL

### Tokenization

In [6]:
import re

def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

data['body_text_tokenized'] = data['body_text_clean'].apply(lambda x: tokenize(x.lower()))

data.head()

label  \
0   ham   
1  spam   
2   ham   
3   ham   
4   ham   

                                                                                             body_text  \
0  I've been searching for the right words to thank you for this breather. I promise i wont take yo...   
1  Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...   
2                                        Nah I don't think he goes to usf, he lives around here though   
3                        Even my brother is not like to speak with me. They treat me like aids patent.   
4                                                                  I HAVE A DATE ON SUNDAY WITH WILL!!   

                                                                                       body_text_clean  \
0  Ive been searching for the right words to thank you for this breather I promise i wont take your...   
1  Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005 Text FA to 87121 to receive e...   
2                                          Nah I dont think he goes to usf he lives around here though   
3                          Even my brother is not like to speak with me They treat me like aids patent   
4                                                                    I HAVE A DATE ON SUNDAY WITH WILL   

                                                                                   body_text_tokenized  
0  [ive, been, searching, for, the, right, words, to, thank, you, for, this, breather, i, promise, ...  
1  [free, entry, in, 2, a, wkly, comp, to, win, fa, cup, final, tkts, 21st, may, 2005, text, fa, to...  
2                            [nah, i, dont, think, he, goes, to, usf, he, lives, around, here, though]  
3         [even, my, brother, is, not, like, to, speak, with, me, they, treat, me, like, aids, patent]  
4                                                           [i, have, a, date, on, sunday, with, will]

In [7]:
'NLP' == 'nlp'

False

## Remove stopwords

Stopwords are commonly-used words like the, but, if, that don't contribute much to the meaning of a sentence. So we want to remove them, to limit the number of tokens Python actually has to look at when building our model.

In [8]:
import nltk

stopwords = nltk.corpus.stopwords.words('english')
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [9]:
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopwords]
    return text

data['body_text_nostop'] = data['body_text_tokenized'].apply(lambda x: remove_stopwords(x))

data.head()

label  \
0   ham   
1  spam   
2   ham   
3   ham   
4   ham   

                                                                                             body_text  \
0  I've been searching for the right words to thank you for this breather. I promise i wont take yo...   
1  Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...   
2                                        Nah I don't think he goes to usf, he lives around here though   
3                        Even my brother is not like to speak with me. They treat me like aids patent.   
4                                                                  I HAVE A DATE ON SUNDAY WITH WILL!!   

                                                                                       body_text_clean  \
0  Ive been searching for the right words to thank you for this breather I promise i wont take your...   
1  Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005 Text FA to 87121 to receive e...   
2                                          Nah I dont think he goes to usf he lives around here though   
3                          Even my brother is not like to speak with me They treat me like aids patent   
4                                                                    I HAVE A DATE ON SUNDAY WITH WILL   

                                                                                   body_text_tokenized  \
0  [ive, been, searching, for, the, right, words, to, thank, you, for, this, breather, i, promise, ...   
1  [free, entry, in, 2, a, wkly, comp, to, win, fa, cup, final, tkts, 21st, may, 2005, text, fa, to...   
2                            [nah, i, dont, think, he, goes, to, usf, he, lives, around, here, though]   
3         [even, my, brother, is, not, like, to, speak, with, me, they, treat, me, like, aids, patent]   
4                                                           [i, have, a, date, on, sunday, with, will]   

                                                                                      body_text_nostop  
0  [ive, searching, right, words, thank, breather, promise, wont, take, help, granted, fulfil, prom...  
1  [free, entry, 2, wkly, comp, win, fa, cup, final, tkts, 21st, may, 2005, text, fa, 87121, receiv...  
2                                                 [nah, dont, think, goes, usf, lives, around, though]  
3                                              [even, brother, like, speak, treat, like, aids, patent]  
4                                                                                       [date, sunday]

So now we have a cleaned column that has been tokenized, we've removed the punctuation and we've removed the stopwords. So that is a very abbreviated look at what a pre-processing pipeline looks like as you're preparing to get your raw text into a format that a machine learning model can actually use. 